In [3]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

In [4]:
def current(B, arrJ, arrC, y): # y is initial phase difference of the whole circuit, B is the magnetic field, arrJ is the location of junctions, arrC is critical current associated with each junction
    curr = 0 # summation of all currents in the entire junction
    limit = int(len(arrJ) / 2) # number of junctions in the SQUID
    numOfSegments = 0
    for n in range(limit):
        numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
        sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
        for i in range(numOfSegments):
            curr += arrC[n] * np.sin(y + (2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment)) * (1/numOfSegments)

    # phase difference evolves according to 2 * pi * B

    # curr += (critical current element in array)(sin(y + (2 * pi * B) * length)
    return curr

def maxCurrent(B, arrayJ, arrayC): # Spits out the maximum current by varying the gauge invariant phase of the left end (free parameter) gamma
    Y=np.linspace(0, 2*np.pi, 150)
    dummyArray=[]
    for gamma in Y:
        dummyArray.append(current(B, arrayJ, arrayC, gamma))
    return max(dummyArray)

def criticalCurrent(density, arrJ):
    criticalCurrents = []
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    for i in range (len(junctionWidths)):
        criticalCurrents.append(junctionWidths[i] * density[i])
    return criticalCurrents, junctionWidths

In [5]:
def cost_function(arr):
    generated_data = []
    model_data_csv = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
    model_i_c = np.array(model_data_csv['I_c'])
    model_B = np.array(model_data_csv['B'])
    criticalCurrents, junctionWidths = criticalCurrent(density=np.ones(len(arr)//2), arrJ=arr)
    for b in model_B:
        generated_data.append(maxCurrent(B=b, arrayJ=arr, arrayC=criticalCurrents))
    diff = model_i_c - generated_data
    for i in range(len(diff)):
        diff[i] = diff[i]**2
    return np.sum(diff)

def optimize_array(n):
    # Initial guess for the sorted array
    initial_guess = np.linspace(0, 1, n)
    initial_guess[0] = 0
    initial_guess[-1] = 1

    # Minimize the cost function
    result = minimize(cost_function, initial_guess, method='BFGS')

    if result.success:
        optimized_array = result.x
        return optimized_array
    else:
        raise ValueError("Optimization failed")


In [6]:
n = 8
optimized_arr = optimize_array(n)

KeyboardInterrupt: 

In [ ]:
print(optimized_arr)